In [34]:
# !pip install bayesian-optimization

In [7]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.tsa.stattools import coint
from statsmodels.regression.linear_model import OLS
import scipy.stats as scs
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as datetime
import itertools
from itertools import product
from bayes_opt import BayesianOptimization
import yfinance as yf

In [2]:
import os
os.chdir('/Users/selena/Desktop/QF2103/Project')
pd.set_option('future.no_silent_downcasting', True)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/selena/Desktop/QF2103/Project'

In [9]:
def clean_data(file):
    raw = pd.read_csv(file, header=1, index_col=0)
    data = raw.iloc[1:, :].copy()
    # reformat and clean dataset
    data.rename_axis("Date", inplace=True)
    data.index = pd.to_datetime(data.index, format="%d/%m/%Y")
    data = data.loc['2024-03-01':] # trading period data
    data.dropna(inplace=True) # last 2 rows NaN values dropped
    # rename all columns according to metrics
    metrics = ["Close", "Dividends", "High", "Low", "Open", "Stock Splits", "Volume"]
    columns = data.columns
    new_columns = {}
    for col in columns:
      if '.' in col:
        stock, suffix = col.split('.')
        metric_index = int(suffix)
        new_columns[col] = f"{stock}_{metrics[metric_index]}"
      else:
        new_columns[col] = f"{col}_Close"
    data.rename(columns=new_columns, inplace=True)
    return data


def benchmark_strategy(data, stock, rf = 0.04):
    df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
    df['Returns'] = np.log(data[stock]/data[stock].shift(1))
    df['Cumulative Returns'] = df['Returns'].cumsum()
    total_return = np.exp(df['Cumulative Returns'].iloc[-1]) - 1
    annualized_return = np.mean(df['Returns']) * 252
    daily_volatility = np.std(df['Returns'])
    annualized_volatility = daily_volatility * np.sqrt(252)
    sharpe_ratio = np.where(annualized_volatility != 0,
                        (annualized_return - rf) / annualized_volatility,
                        0)

    metrics = {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio}
    
    return metrics, df

### Trading period data (17 Jan 2024 to 16 January 2025)

In [12]:
file = 'Trading_Project_Data.csv'
data = clean_data(file).iloc[:, :20].copy() # extract close prices from cleaned data
stocks = list(set([col.split("_")[0] for col in data.columns]))
stocks.sort()
for stock in stocks:
    data.rename(columns={f'{stock}_Close': f'{stock}'}, inplace=True)
data

,AAPL,AMZN,BA,CAT,CVX,GOOGL,GS,JNJ,JPM,KO,MCD,MSFT,NKE,NVDA,PFE,SOFI,TSLA,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
2024-03-01,179.012268,178.220001,200.000000,331.704254,148.097229,136.644638,381.606262,158.338089,181.148910,57.765736,285.466309,413.178650,100.519699,82.255676,25.435343,8.990000,202.639999,481.983704,58.064034,103.299927
2024-03-04,174.468719,177.580002,200.539993,333.625275,144.288452,132.868347,385.686829,156.111267,182.507858,58.037441,285.672455,412.601898,98.210945,85.212837,24.765739,8.580000,188.139999,474.441773,58.597641,101.855453
2024-03-05,169.506653,174.119995,201.139999,328.581268,144.976547,132.190796,384.369232,156.238251,184.336060,57.756035,287.243530,400.400452,96.997368,85.943665,24.947489,7.270000,180.740005,465.856171,59.328873,103.104736
2024-03-06,168.510254,173.509995,201.000000,330.650147,143.755402,130.925369,383.051666,155.622940,185.294174,57.785149,287.626404,399.843597,96.326447,88.679016,26.009287,7.400000,176.539993,465.314667,59.852596,104.207611
2024-03-07,168.390686,176.820007,203.029999,335.171997,144.763321,133.894623,381.930725,155.163895,183.671265,57.678410,285.839356,406.854218,96.878975,92.647072,25.626657,7.470000,178.649994,471.399383,59.645084,104.793213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-10,236.850006,218.940002,172.000000,349.717926,153.139999,192.039993,560.000000,142.059998,239.869995,61.070000,282.309998,418.950012,71.199997,135.910004,26.719999,14.150000,394.739990,520.690002,93.000000,106.540001
2025-01-13,234.399994,218.460007,170.570007,361.175934,155.350006,191.009995,562.950012,144.470001,244.210007,61.650002,283.630005,417.190002,72.080002,133.229996,26.799999,14.070000,403.309998,541.140015,91.529999,109.290001
2025-01-14,233.279999,217.759995,167.020004,370.212799,156.899994,189.660004,571.530029,144.750000,247.470001,62.040001,281.579987,415.670013,71.169998,131.759995,26.410000,14.500000,396.359985,543.739990,90.790001,109.720001


### Testing data (1 March 2023 to 16 January 2024)

In [15]:
# 1 year before testing data
test = yf.Tickers(stocks)
test_data = test.history(start = '2023-01-23', end = '2024-01-16')
columns_to_drop = ['High', 'Low', 'Open', 'Volume', 'Dividends', 'Stock Splits']
test_data.drop(columns=columns_to_drop, inplace=True)
# test_data = test_data.swaplevel(0,1,axis=1).sort_index(axis=1)
test_data.columns = test_data.columns.droplevel(0)

[*********************100%***********************]  20 of 20 completed


### pairs trading strategy

In [17]:
def cointegration_test(data, threshold):
    stock_pairs = itertools.combinations(data.columns, 2) # get all pairs of stocks
    cointegrated_pairs = []

    for stock1, stock2 in stock_pairs:
        X = sm.add_constant(data[stock1]) # regress stock2 (DV) on stock1 (IV)
        model = OLS(data[stock2], X).fit()
        residuals = data[stock2] - model.predict() # errors from regression (spread)
        # test residuals for stationarity using a unit root test -> Augmented Dickey-Fuller test
        adf_test = adfuller(residuals)
        p_value = adf_test[1]
        if p_value < threshold: # residuals are stationary, stock1 and stock2 are cointegrated
            cointegrated_pairs.append((stock1, stock2))
    return cointegrated_pairs


def pairs_strat(data, pair, window, entry_threshold, exit_threshold, rf=0.04):
    stock1, stock2 = pair
    
    # compute zscore for the pair
    X = sm.add_constant(data[stock1])
    model = sm.OLS(data[stock2], X).fit()
    spread = data[stock2] - model.predict(X) # calculate the spread as residuals
    # rolling mean and std of spread
    spread_mean = spread.rolling(window=window).mean()
    spread_std = spread.rolling(window=window).std()
    zscore = (spread - spread_mean) / spread_std
    zscore.dropna(inplace=True)

    # apply pairs trading strat
    # initialise DataFrame to store positions and returns for the pair
    pair_df = pd.DataFrame(index=zscore.index, columns=[f'{stock1} Position', f'{stock2} Position', f'{stock1} Returns', f'{stock2} Returns'])

    # initialise positions and returns as 0 both for stocks for the start
    pair_df.iloc[:, :] = 0

    # generate daily entry and exit signals (positions)
    for i in pair_df.index[1:]:
        current_zscore = zscore.loc[i]
            
        # zscore < -entry_threshold and no existing long position for stock 1
        if (current_zscore < -entry_threshold) & (pair_df[f'{stock1} Position'].shift(1).at[i] == 0):
            pair_df.loc[i, f'{stock1} Position'] = 1  # long stock 1
            pair_df.loc[i, f'{stock2} Position'] = -0.5 # short stock 2

        # zscore > entry_threshold and no existing short position for stock 2
        elif (current_zscore > entry_threshold) & (pair_df[f'{stock2} Position'].shift(1).at[i] == 0):
            pair_df.loc[i, f'{stock1} Position'] = -0.5  # short stock 1
            pair_df.loc[i, f'{stock2} Position'] = 1   # long stock 2

        # -exit_threshold < zscore < exit_threshold

        elif abs(current_zscore) < exit_threshold:
            pair_df.loc[i, f'{stock1} Position'] = 0  # exit position in stock 1
            pair_df.loc[i, f'{stock2} Position'] = 0  # exit position in stock 2
    

        # -entry_threshold < zscore < -exit_threshold or exit_threshold < zscore < entry_threshold
        else:
            pair_df.loc[i, f'{stock1} Position'] = pair_df[f'{stock1} Position'].shift(1).at[i] # maintain existing position in stock 1
            pair_df.loc[i, f'{stock2} Position'] = pair_df[f'{stock2} Position'].shift(1).at[i] # maintain existing position in stock 2

        # calculate daily (log) returns for each stock based on the positions
        pair_df.loc[i, f'{stock1} Returns'] = (np.log(data.loc[i, stock1] / data[stock1].shift(1).at[i])) * pair_df[f'{stock1} Position'].shift(1).loc[i]
        pair_df.loc[i, f'{stock2} Returns'] = (np.log(data.loc[i, stock2] / data[stock2].shift(1).at[i])) * pair_df[f'{stock2} Position'].shift(1).loc[i]

    pair_df = pair_df.iloc[:-1, :]

    if pair_df.empty or len(pair_df) < window:
        return {'total_return': 0, 'other_metrics': None}, pair_df
        
    # calculate combined (log) return, (log) cumulative return and total (simple) return
    pair_df['combined_returns'] =  pair_df[f'{stock1} Returns'] + pair_df[f'{stock2} Returns']
    pair_df['combined_returns'] = pair_df['combined_returns'].astype(float)
    pair_df['cumulative_combined_returns'] = pair_df['combined_returns'].cumsum()
    total_return = np.exp(pair_df['cumulative_combined_returns'].iloc[-1]) - 1
    annualized_return = np.mean(pair_df['combined_returns']) * 252
    daily_volatility = np.std(pair_df['combined_returns'])
    annualized_volatility = daily_volatility * np.sqrt(252)

    sharpe_ratio = np.where(annualized_volatility != 0,
                        (annualized_return - rf) / annualized_volatility,
                        0)

    metrics = {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio}
    
    return metrics, pair_df

In [20]:
def plot_pairs(data, pair_df, pair, entry_threshold, exit_threshold):
    stock1, stock2 = pair
    
    stock1_log_rets = np.log(data[stock1] / data[stock1].shift(1)).fillna(0)
    stock2_log_rets = np.log(data[stock2] / data[stock2].shift(1)).fillna(0)
    
    # plot (simple) cumulative returns 
    np.exp(pair_df['cumulative_combined_returns']).plot()
    plt.title(f'Cumulative returns for {stock1}-{stock2} pairs')
    plt.plot(np.exp(stock1_log_rets.cumsum()), label=f'{stock1} Benchmark')
    plt.plot(np.exp(stock2_log_rets.cumsum()), label=f'{stock2} Benchmark')
    plt.legend()
    plt.show()

### bayes opt for pairs strategy

### non rolling (opt total returns)

In [24]:
# find parameters values for window, entry and exit for pairs strat that maximises the total returns of the strat (for each pair)
def pairs_bayes_opt(data, pair, window_range = (5, 50), entry_range=(1.5, 3), exit_range=(0, 1), init_points = 5, n_iter = 50):
    pbounds = {'window': window_range, 'entry_threshold': entry_range, 'exit_threshold': exit_range}
    
    def objective(window, entry_threshold, exit_threshold):
        metrics, _ = pairs_strat(data, pair, window=int(window), entry_threshold=entry_threshold, exit_threshold=exit_threshold)
        return metrics['total_return']

    # perform bayes opt
    optimizer = BayesianOptimization(f=objective, # target function to maximise
                                     pbounds=pbounds, # params bounds
                                     random_state=42)
    
    # optimize the params
    optimizer.maximize(init_points=init_points, n_iter=n_iter)
    
    # extract optimized params and total return
    best_window = int(optimizer.max['params']['window']) # round to integer
    best_entry = optimizer.max['params']['entry_threshold']
    best_exit = optimizer.max['params']['exit_threshold']
    best_total_return = optimizer.max['target']
    
    return best_window, best_entry, best_exit, best_total_return

### test data codes

In [27]:
test_cointegrated_pairs = cointegration_test(test_data, threshold=0.05)
test_cointegrated_pairs

[('AAPL', 'BA'),
 ('AAPL', 'CVX'),
 ('AAPL', 'XOM'),
 ('AMZN', 'CVX'),
 ('AMZN', 'MSFT'),
 ('AMZN', 'NVDA'),
 ('AMZN', 'XOM'),
 ('BA', 'CVX'),
 ('BA', 'XOM'),
 ('CAT', 'SOFI'),
 ('CAT', 'TSLA'),
 ('CVX', 'XOM'),
 ('GOOGL', 'NVDA'),
 ('GOOGL', 'WMT'),
 ('GS', 'XOM'),
 ('JNJ', 'SOFI'),
 ('JPM', 'MSFT'),
 ('JPM', 'XOM'),
 ('KO', 'XOM'),
 ('MCD', 'XOM'),
 ('MSFT', 'XOM'),
 ('NKE', 'SOFI'),
 ('NKE', 'TSLA'),
 ('NVDA', 'WMT'),
 ('PFE', 'UNH'),
 ('SOFI', 'TSLA')]

In [28]:
test_pairs_all_best_params = {}
test_otpimized_pairs_strat_df = pd.DataFrame(index=test_cointegrated_pairs, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])
for pair in test_cointegrated_pairs:
    best_window, best_entry, best_exit, best_total_return = pairs_bayes_opt(test_data, pair, window_range = (5, 50), entry_range=(1.5, 3), exit_range=(0, 1), init_points = 5, n_iter = 50)
    metrics, _ = pairs_strat(test_data, pair, window=best_window, entry_threshold=best_entry, exit_threshold=best_exit)
    test_otpimized_pairs_strat_df.loc[[pair], 'Total Returns'] = metrics['total_return']
    test_otpimized_pairs_strat_df.loc[[pair], 'Annualized Return'] = metrics['annualized_return']
    test_otpimized_pairs_strat_df.loc[[pair], 'Annualized Volatility'] = metrics['annualized_volatility']
    test_otpimized_pairs_strat_df.loc[[pair], 'Sharpe Ratio'] = metrics['sharpe_ratio']
    test_pairs_all_best_params[pair] = {"best_entry": best_entry,
                                   "best_exit": best_exit,
                                   "best_total_return": best_total_return}
    print(f'{pair} Optimal Parameters are entry: {best_entry:.3f}, exit: {best_exit:.3f}')

|   iter    |  target   | entry_... | exit_t... |  window   |
-------------------------------------------------------------
| 1         | 0.1563    | 2.062     | 0.9507    | 37.94     |
| 2         | 0.1239    | 2.398     | 0.156     | 12.02     |
| 3         | 0.2077    | 1.587     | 0.8662    | 32.05     |
| 4         | -0.04841  | 2.562     | 0.02058   | 48.65     |
| 5         | 0.05511   | 2.749     | 0.2123    | 13.18     |
| 6         | 0.2317    | 1.687     | 0.7966    | 33.88     |
| 7         | 0.1858    | 1.551     | 0.04214   | 7.456     |
| 8         | -0.03602  | 2.886     | 0.1487    | 26.32     |
| 9         | 0.0       | 2.982     | 0.8514    | 5.235     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 10        | 0.0       | 2.993     | 0.1154    | 33.05     |
| 11        | 0.07619   | 1.998     | 0.6799    | 48.34     |
| 12        | 0.2093    | 1.528     | 0.8739    | 33.52     |
| 13        | 0.2046    | 1.513     | 0.7649    | 34.78     |
| 14        | 0.2577    | 1.56      | 0.976     | 30.75     |
| 15        | 0.1452    | 1.59      | 0.01752   | 30.16     |
| 16        | 0.2238    | 2.635     | 0.9681    | 30.78     |
| 17        | 0.4528    | 1.797     | 0.1979    | 9.314     |
| 18        | 0.09194   | 1.523     | 0.9389    | 9.555     |
| 19        | 0.2622    | 2.419     | 0.06151   | 24.12     |
| 20        | 0.01139   | 2.105     | 0.1493    | 9.24      |
| 21        | -0.02042  | 2.919     | 0.4514    | 45.96     |
| 22        | 0.1128    | 2.521     | 0.3538    | 11.49     |
| 23        | 0.1184    | 2.004     | 0.05813   | 44.53     |
| 24        | 0.05873   | 2.82      | 0.8377    | 19.43     |
| 25        | 0.3075    | 1.994     | 0.9631    | 27.74     |
| 26    

/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 11        | 0.3012    | 1.518     | 0.4497    | 9.988     |
| 12        | 0.2036    | 1.723     | 0.9968    | 9.252     |
| 13        | -0.1186   | 2.388     | 0.05386   | 9.974     |
| 14        | 0.1555    | 1.518     | 0.3284    | 9.567     |
| 15        | 0.1045    | 1.779     | 0.42      | 8.984     |
| 16        | 0.1098    | 1.962     | 0.8611    | 9.223     |
| 17        | 0.118     | 1.57      | 0.6799    | 10.15     |
| 18        | 0.3606    | 1.902     | 0.2553    | 27.7      |
| 19        | 0.1471    | 2.419     | 0.06151   | 24.12     |
| 20        | 0.4456    | 2.07      | 0.1942    | 27.73     |
| 21        | 0.2758    | 1.55      | 0.4697    | 9.971     |
| 22        | 0.2409    | 2.227     | 0.3894    | 27.86     |
| 23        | -0.1726   | 2.004     | 0.05813   | 44.53     |
| 24        | 0.2347    | 1.991     | 0.007996  | 27.89     |
| 25        | 0.1426    | 2.169     | 0.09913   | 27.53     |
| 26        | 0.4389    | 1.997     | 0.2562    | 27.79     |
| 27    

...

| 47        | 0.0       | 2.856     | 0.1117    | 14.76     |
| 48        | 0.05187   | 1.663     | 0.01488   | 5.093     |
| 49        | 0.1702    | 2.86      | 0.008482  | 21.24     |
| 50        | 0.1025    | 2.88      | 0.1244    | 21.17     |
| 51        | -0.3057   | 2.64      | 0.6107    | 38.92     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 52        | 0.0       | 2.992     | 0.02927   | 21.3      |
| 53        | -0.2658   | 1.654     | 0.9473    | 7.871     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 54        | 0.0       | 2.93      | 0.2249    | 20.94     |
| 55        | -0.09011  | 2.777     | 0.0409    | 21.35     |
('SOFI', 'TSLA') Optimal Parameters are entry: 1.625, exit: 0.017


In [30]:
test_otpimized_pairs_strat_df = test_otpimized_pairs_strat_df.sort_values(by='Total Returns', ascending=False)
test_otpimized_pairs_strat_df

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
"(JNJ, SOFI)",0.968355,0.726187,0.464764,1.476421
"(AMZN, XOM)",0.946289,0.766269,0.292029,2.486977
"(NKE, SOFI)",0.779278,0.610102,0.358363,1.590849
"(AMZN, CVX)",0.742966,0.616776,0.280571,2.055725
"(BA, CVX)",0.688704,0.66686,0.156632,4.002108
"(AAPL, CVX)",0.64501,0.527026,0.192675,2.527701
"(NVDA, WMT)",0.548372,0.470835,0.327507,1.315497
"(GS, XOM)",0.544497,0.542297,0.178129,2.819841
"(JPM, MSFT)",0.532153,0.490967,0.218921,2.059956
"(AAPL, BA)",0.502209,0.432692,0.189808,2.068892


In [31]:
selected_stocks = list(set([stock for pair in test_cointegrated_pairs for stock in pair]))
test_benchmark_strat_df = pd.DataFrame(index=selected_stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])

for stock in selected_stocks:
    metrics, _ = benchmark_strategy(test_data, stock)
    
    test_benchmark_strat_df.loc[stock, 'Total Returns'] = metrics['total_return']
    test_benchmark_strat_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    test_benchmark_strat_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    test_benchmark_strat_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

test_benchmark_strat_df = test_benchmark_strat_df.sort_values(by='Total Returns', ascending=False)
test_benchmark_strat_df

/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays 

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
NVDA,1.851783,1.077886,0.456251,2.274814329938806
MSFT,0.61561,0.493418,0.240662,1.8840491870700762
AMZN,0.585521,0.474082,0.320973,1.3523930586880877
TSLA,0.522713,0.432508,0.511584,0.7672407408772831
GOOGL,0.429502,0.367535,0.299436,1.0938417970812815
SOFI,0.344123,0.304191,0.653402,0.40433198410969445
AAPL,0.324918,0.289389,0.19921,1.2518922129152963
JPM,0.266688,0.24316,0.202744,1.0020536161175664
CAT,0.167189,0.159015,0.283609,0.41964480901025425
WMT,0.148209,0.142152,0.15365,0.6648377598756733


### trading data codes

In [33]:
cointegrated_pairs = cointegration_test(data, threshold=0.05)
cointegrated_pairs

[('AAPL', 'NKE'),
 ('AAPL', 'XOM'),
 ('AMZN', 'GOOGL'),
 ('AMZN', 'XOM'),
 ('BA', 'CAT'),
 ('BA', 'MCD'),
 ('BA', 'UNH'),
 ('BA', 'XOM'),
 ('CAT', 'MCD'),
 ('CAT', 'XOM'),
 ('CVX', 'KO'),
 ('CVX', 'PFE'),
 ('GOOGL', 'XOM'),
 ('GS', 'JPM'),
 ('GS', 'WMT'),
 ('JNJ', 'KO'),
 ('JNJ', 'UNH'),
 ('JNJ', 'XOM'),
 ('JPM', 'SOFI'),
 ('JPM', 'TSLA'),
 ('JPM', 'WMT'),
 ('KO', 'PFE'),
 ('MCD', 'XOM'),
 ('MSFT', 'XOM'),
 ('SOFI', 'XOM'),
 ('TSLA', 'XOM'),
 ('UNH', 'XOM')]

In [34]:
pairs_all_best_params = {}
otpimized_pairs_strat_df = pd.DataFrame(index=cointegrated_pairs, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])
for pair in cointegrated_pairs:
    best_window, best_entry, best_exit, best_total_return = pairs_bayes_opt(data, pair, window_range = (5, 50), entry_range=(1.5, 3), exit_range=(0, 1), init_points = 5, n_iter = 50)
    metrics, _ = pairs_strat(data, pair, window=best_window, entry_threshold=best_entry, exit_threshold=best_exit)
    otpimized_pairs_strat_df.loc[[pair], 'Total Returns'] = metrics['total_return']
    otpimized_pairs_strat_df.loc[[pair], 'Annualized Return'] = metrics['annualized_return']
    otpimized_pairs_strat_df.loc[[pair], 'Annualized Volatility'] = metrics['annualized_volatility']
    otpimized_pairs_strat_df.loc[[pair], 'Sharpe Ratio'] = metrics['sharpe_ratio']
    pairs_all_best_params[pair] = {"best_entry": best_entry,
                                   "best_exit": best_exit,
                                   "best_total_return": best_total_return}
    print(f'{pair} Optimal Parameters are entry: {best_entry:.3f}, exit: {best_exit:.3f}')

|   iter    |  target   | entry_... | exit_t... |  window   |
-------------------------------------------------------------
| 1         | -0.2747   | 2.062     | 0.9507    | 37.94     |
| 2         | -0.1081   | 2.398     | 0.156     | 12.02     |
| 3         | -0.328    | 1.587     | 0.8662    | 32.05     |
| 4         | 0.0       | 2.562     | 0.02058   | 48.65     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 5         | 0.06356   | 2.749     | 0.2123    | 13.18     |
| 6         | -0.04538  | 2.716     | 0.07585   | 14.09     |
| 7         | 0.04829   | 2.117     | 0.972     | 13.31     |
| 8         | 0.0       | 2.885     | 0.7326    | 47.04     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 9         | -0.2099   | 1.551     | 0.01743   | 47.1      |
| 10        | 0.1073    | 2.969     | 0.9962    | 13.1      |


...

| 15        | 0.0       | 2.948     | 0.04546   | 23.41     |
| 16        | -0.2484   | 1.612     | 0.9013    | 44.16     |
| 17        | 0.3083    | 1.543     | 0.1481    | 9.474     |
| 18        | -0.06448  | 1.523     | 0.9389    | 9.555     |
| 19        | -0.01502  | 2.4       | 0.1279    | 12.04     |
| 20        | 0.03502   | 1.923     | 0.05025   | 9.422     |
| 21        | 0.3831    | 1.524     | 0.2584    | 9.604     |
| 22        | 0.4936    | 1.542     | 0.1906    | 9.852     |
| 23        | -0.05073  | 1.502     | 0.02384   | 10.27     |
| 24        | 0.3293    | 1.595     | 0.3992    | 9.897     |
| 25        | 0.2853    | 1.684     | 0.1453    | 9.779     |
| 26        | 0.1098    | 1.502     | 0.07465   | 9.657     |
| 27        | -0.1632   | 2.29      | 0.2212    | 47.12     |
| 28        | -0.2055   | 1.605     | 0.7807    | 44.26     |
| 29        | 0.4606    | 1.68      | 0.1788    | 9.764     |
| 30        | 0.2494    | 1.785     | 0.3485    | 9.664     |
| 31    

/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 42        | 0.0       | 2.65      | 0.2231    | 14.23     |
| 43        | -0.1578   | 2.232     | 0.8013    | 44.41     |
| 44        | -0.03692  | 2.317     | 0.3864    | 10.09     |
| 45        | 0.2044    | 1.587     | 0.5173    | 9.464     |
| 46        | 0.5004    | 1.588     | 0.3701    | 9.315     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/1122759907.py:82: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 47        | 0.0       | 2.856     | 0.1117    | 14.76     |
| 48        | 0.3906    | 1.568     | 0.2291    | 9.334     |
| 49        | 0.1034    | 1.582     | 0.5533    | 9.215     |
| 50        | 0.3712    | 1.523     | 0.2717    | 9.204     |
| 51        | -0.13     | 2.64      | 0.6107    | 38.92     |
| 52        | 0.1016    | 1.768     | 0.3804    | 9.281     |
| 53        | -0.001241 | 1.654     | 0.9473    | 7.871     |
| 54        | 0.4679    | 1.696     | 0.3174    | 9.918     |
| 55        | 0.1831    | 1.845     | 0.4575    | 10.02     |
('UNH', 'XOM') Optimal Parameters are entry: 1.588, exit: 0.370


In [35]:
otpimized_pairs_strat_df = otpimized_pairs_strat_df.sort_values(by='Total Returns', ascending=False)
otpimized_pairs_strat_df

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
"(SOFI, XOM)",1.333442,0.988569,0.407331,2.32874
"(JPM, TSLA)",1.260285,0.983271,0.597114,1.579717
"(AAPL, XOM)",0.56706,0.554896,0.225553,2.282815
"(AAPL, NKE)",0.537446,0.508878,0.249633,1.878265
"(BA, XOM)",0.533557,0.518042,0.232452,2.056523
"(AMZN, XOM)",0.510808,0.52785,0.254342,1.918083
"(UNH, XOM)",0.500367,0.482259,0.200963,2.200703
"(CVX, KO)",0.433182,0.479863,0.151487,2.903641
"(BA, MCD)",0.432368,0.511587,0.165351,2.852037
"(JPM, SOFI)",0.416543,0.452327,0.335764,1.228025


In [36]:
selected_stocks = list(set([stock for pair in cointegrated_pairs for stock in pair]))
benchmark_strat_df = pd.DataFrame(index=selected_stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])

for stock in selected_stocks:
    metrics, _ = benchmark_strategy(data, stock)
    
    benchmark_strat_df.loc[stock, 'Total Returns'] = metrics['total_return']
    benchmark_strat_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    benchmark_strat_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    benchmark_strat_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

benchmark_strat_df = benchmark_strat_df.sort_values(by='Total Returns', ascending=False)
benchmark_strat_df

/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays 

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
TSLA,1.042144,0.817855,0.651925,1.193165363819431
SOFI,0.813126,0.681605,0.526906,1.2176851222520606
GS,0.606342,0.542899,0.271529,1.8520994449456636
WMT,0.572402,0.518438,0.178516,2.680077539635155
GOOGL,0.411764,0.394999,0.275298,1.289506528453523
JPM,0.403652,0.388398,0.246083,1.4157722521803078
AAPL,0.275108,0.278381,0.236102,1.0096521343371667
AMZN,0.238133,0.244674,0.277033,0.7388070234461415
CAT,0.143066,0.153164,0.26459,0.42769481524586483
XOM,0.077639,0.085648,0.19088,0.23914664208098269


## bollinger bands strategy

In [38]:
def bollinger_bands_strat(data, stock, std, window, rf=0.04):
    stock_df = data[[f'{stock}']].copy()
    stock_df.rename(columns={f'{stock}': 'Close'}, inplace=True)

    # calculate log returns
    stock_df['Returns'] = np.log(stock_df['Close'] / stock_df['Close'].shift(1)).fillna(0)
    
    # calculate SMA and rolling std on close price
    stock_df['SMA'] = stock_df['Close'].rolling(window=window).mean()
    stock_df['STD'] = stock_df['Close'].rolling(window=window).std()
    stock_df.dropna(inplace=True)

    # calculate the upper and lower bollinger bands
    stock_df['UB'] = stock_df['SMA'] + (stock_df['STD'] * std)
    stock_df['LB'] = stock_df['SMA'] - (stock_df['STD'] * std)
        
    # calculate the position (buy and sell signals)
    stock_df['Position'] = np.where(stock_df['Close'] > stock_df['UB'], -0.5, 
                                    np.where(stock_df['Close'] < stock_df['LB'], 1, 0))
    # calculate strategy returns
    stock_df['Strat_Returns'] = stock_df['Returns'] * stock_df['Position'].shift(1)

    # total simple return
    total_return = np.exp(stock_df['Strat_Returns'].sum()) - 1


    annualized_return = np.mean(np.exp(stock_df['Strat_Returns']) - 1) * 252
    daily_volatility = np.std(np.exp(stock_df['Strat_Returns']) - 1)
    annualized_volatility = daily_volatility * np.sqrt(252)
    sharpe_ratio = np.where(annualized_volatility != 0,
                        (annualized_return - rf) / annualized_volatility,
                        0)

    metrics = {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio}
    
    return metrics, stock_df

In [39]:
def plot_bollinger(stock_df, stock):
    # Plot the (simple) cumulative returns of the strategy compared to the benchmark
    plt.figure(figsize=(12, 6))
    plt.plot(np.exp(stock_df['Strat_Returns'].cumsum()), label='Mean Reversion Strategy')
    plt.plot(np.exp(stock_df['Returns'].cumsum()), label='Benchmark')
    plt.legend()
    plt.title(f'{stock} Cumulative Simple Returns')
    plt.show()

    # trading actions
    # entry signals
    stock_df['Entry_Long'] = (stock_df['Position'].shift(1) == 0) & (stock_df['Position'] == 1) # 0 -> 1
    stock_df['Entry_Short'] = (stock_df['Position'].shift(1) == 0) & (stock_df['Position'] == -0.5)  # 0 -> -0.5
    # exit Signals
    stock_df['Exit_Long'] = (stock_df['Position'].shift(1) == 1) & (stock_df['Position'] == 0)  # 1 -> 0
    stock_df['Exit_Short'] = (stock_df['Position'].shift(1) == -0.5) & (stock_df['Position'] == 0)  # -0.5 -> 0

    # plot close price series, SMA, bollinger bands and trading actions
    plt.figure(figsize=(12, 6))
    plt.plot(stock_df['UB'], label = 'UB', color = 'black')
    plt.plot(stock_df['LB'], label = 'LB', color = 'black')
    plt.plot(stock_df['Close'], label = 'Close', color = 'blue')
    plt.plot(stock_df[stock_df.Entry_Long].index, stock_df['LB'][stock_df.Entry_Long], '^', color = 'g', label = 'Buy Signal', markersize = 8)
    plt.plot(stock_df[stock_df.Entry_Short].index, stock_df['UB'][stock_df.Entry_Short], 'v', color = 'r', label = 'Sell Signal', markersize = 8)
    plt.plot(stock_df[stock_df.Exit_Long].index, stock_df['LB'][stock_df.Exit_Long], 'x', color = 'g', label = 'Exit Long', markersize = 8)
    plt.plot(stock_df[stock_df.Exit_Short].index, stock_df['UB'][stock_df.Exit_Short], 'x', color = 'r', label = 'Exit Short', markersize = 8)
    plt.title(f'{stock} Bollinger Bands with Trading Actions')
    plt.legend(loc=1)
    plt.show()

### bayes opt for bollinger bands strat

### non rolling (opt total returns)

In [42]:
# find parameters values for std and window for bollinger bands strat that maximises the total returns of the strat (for each stock)
def bollinger_bayes_opt(data, stock, std_range=(1.0, 3.5), window_range=(10, 50), init_points = 5, n_iter = 20):
    # bounds for the params in the opt process
    pbounds = {'std': std_range, 'window': window_range}

    def objective(std, window): # objective function for bayes opt
        metrics, _ = bollinger_bands_strat(data, stock, std, int(window))
        return metrics['total_return']

    # perform bayes opt
    optimizer = BayesianOptimization(f=objective,  # target function to maximise
                                     pbounds=pbounds,  # params bounds
                                     random_state=42)

    # optimize the params
    optimizer.maximize(init_points=init_points, n_iter=n_iter)
    
    # extract optimized params and total return
    best_std = optimizer.max['params']['std']
    best_window = int(optimizer.max['params']['window'])  # round to integer
    best_total_return = optimizer.max['target']
    
    return best_std, best_window, best_total_return

### test data codes

In [44]:
test_bollinger_all_best_params = {}

test_otpimized_bollinger_stocks_df = pd.DataFrame(index=stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])
for stock in stocks:
    best_std, best_window, best_total_return = bollinger_bayes_opt(test_data, stock, std_range=(1.0, 3.0), window_range=(10, 50), init_points = 5, n_iter = 50)
    metrics, _ = bollinger_bands_strat(test_data, stock, best_std, best_window)
    test_otpimized_bollinger_stocks_df.loc[stock, 'Total Returns'] = metrics['total_return']
    test_otpimized_bollinger_stocks_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    test_otpimized_bollinger_stocks_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    test_otpimized_bollinger_stocks_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

    test_bollinger_all_best_params[stock] = {"best_std": best_std,
                                             "best_window": best_window,
                                             "best_total_return": best_total_return}
    
    print(f'{stock} Optimal Parameters are std: {best_std:.3f}, window: {best_window}')

|   iter    |  target   |    std    |  window   |
-------------------------------------------------
| 1         | -0.01391  | 1.749     | 48.03     |
| 2         | 0.02313   | 2.464     | 33.95     |
| 3         | -0.0451   | 1.312     | 16.24     |
| 4         | -0.008449 | 1.116     | 44.65     |
| 5         | 0.02352   | 2.202     | 38.32     |
| 6         | -0.004176 | 1.012     | 28.57     |
| 7         | -0.01798  | 1.002     | 36.02     |
| 8         | -0.03551  | 1.203     | 30.87     |
| 9         | -0.004176 | 1.011     | 28.55     |
| 10        | 0.0       | 2.986     | 39.16     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 11        | 0.007     | 1.272     | 38.81     |
| 12        | 0.0       | 2.957     | 34.92     |
| 13        | 0.0       | 2.965     | 33.05     |
| 14        | -0.04681  | 1.577     | 33.85     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 15        | 0.0       | 2.979     | 33.98     |
| 16        | -0.000697 | 1.261     | 38.82     |
| 17        | 0.03297   | 2.277     | 38.32     |
| 18        | 0.03297   | 2.295     | 38.43     |
| 19        | 0.007427  | 2.399     | 38.35     |
| 20        | -0.004459 | 1.41      | 38.77     |
| 21        | 0.02888   | 2.251     | 38.23     |
| 22        | 0.03061   | 2.154     | 38.47     |
| 23        | 0.005312  | 2.638     | 37.23     |
| 24        | 0.03647   | 2.004     | 36.88     |
| 25        | 0.01712   | 2.08      | 36.94     |
| 26        | 0.03303   | 2.258     | 38.55     |
| 27        | -0.003433 | 1.88      | 36.88     |
| 28        | 0.03061   | 2.151     | 38.61     |
| 29        | -0.02249  | 2.296     | 14.69     |
| 30        | 0.02595   | 2.128     | 36.74     |
| 31        | 0.03402   | 2.03      | 36.77     |
| 32        | -0.02147  | 1.726     | 12.63     |
| 33        | -0.02001  | 2.636     | 18.63     |
| 34        | 0.0       | 2.84      | 41.97     |


...

| 51        | 0.0       | 2.981     | 38.81     |
| 52        | 0.02072   | 1.042     | 19.47     |
| 53        | 0.06832   | 1.589     | 37.38     |
| 54        | 0.004299  | 2.555     | 45.44     |
| 55        | 0.07458   | 1.069     | 32.7      |
WMT Optimal Parameters are std: 1.054, window: 38
|   iter    |  target   |    std    |  window   |
-------------------------------------------------
| 1         | 0.02449   | 1.749     | 48.03     |
| 2         | 0.007762  | 2.464     | 33.95     |
| 3         | 0.05777   | 1.312     | 16.24     |
| 4         | 0.2741    | 1.116     | 44.65     |
| 5         | 0.03607   | 2.202     | 38.32     |
| 6         | 0.2283    | 1.08      | 44.68     |
| 7         | 0.2845    | 1.208     | 44.52     |
| 8         | 0.1689    | 1.442     | 44.65     |
| 9         | 0.2505    | 1.009     | 44.26     |
| 10        | 0.293     | 1.273     | 44.08     |
| 11        | 0.2502    | 1.044     | 43.87     |
| 12        | 0.1442    | 1.383     | 43.82     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 36        | 0.2662    | 1.386     | 40.49     |
| 37        | 0.2713    | 1.041     | 40.16     |
| 38        | 0.1767    | 1.501     | 40.04     |
| 39        | 0.2414    | 1.009     | 40.53     |
| 40        | 0.126     | 1.299     | 28.04     |
| 41        | 0.2717    | 1.003     | 39.65     |
| 42        | -0.0254   | 2.771     | 16.0      |
| 43        | -0.01883  | 2.247     | 22.29     |
| 44        | 0.341     | 1.072     | 39.17     |
| 45        | -0.05846  | 2.719     | 19.69     |
| 46        | -0.01971  | 2.61      | 11.14     |
| 47        | 0.2208    | 1.45      | 39.28     |
| 48        | 0.006753  | 1.759     | 47.64     |
| 49        | 0.2414    | 1.032     | 38.76     |
| 50        | 0.08006   | 1.954     | 40.63     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 51        | 0.0       | 2.997     | 30.55     |
| 52        | 0.2487    | 1.023     | 37.98     |
| 53        | 0.2342    | 1.004     | 37.4      |
| 54        | 0.2533    | 1.428     | 37.74     |
| 55        | 0.3239    | 1.338     | 38.25     |
XOM Optimal Parameters are std: 1.072, window: 39


In [45]:
test_otpimized_bollinger_stocks_df = test_otpimized_bollinger_stocks_df.sort_values(by='Total Returns', ascending=False)
test_otpimized_bollinger_stocks_df

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
SOFI,0.661432,0.660806,0.281885,2.2023392344528636
XOM,0.34098,0.367304,0.141146,2.318908613986301
AMZN,0.304937,0.300326,0.11064,2.3529140290297454
GOOGL,0.207812,0.221518,0.11962,1.517459403414285
MSFT,0.188625,0.197571,0.128935,1.222092529882047
CVX,0.167788,0.183802,0.135363,1.0623409665279904
PFE,0.159964,0.197366,0.137974,1.1405483309201017
UNH,0.15314,0.161851,0.108007,1.1281764625432966
NVDA,0.144295,0.171939,0.232917,0.5664624639271367
CAT,0.134691,0.140983,0.092739,1.08889046885413


In [46]:
test_benchmark_boll_strat_df = pd.DataFrame(index=selected_stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])

for stock in stocks:
    metrics, _ = benchmark_strategy(test_data, stock)
    
    test_benchmark_boll_strat_df.loc[stock, 'Total Returns'] = metrics['total_return']
    test_benchmark_boll_strat_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    test_benchmark_boll_strat_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    test_benchmark_boll_strat_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

test_benchmark_boll_strat_df = test_benchmark_boll_strat_df.sort_values(by='Total Returns', ascending=False)
test_benchmark_boll_strat_df

/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays 

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
NVDA,1.851783,1.077886,0.456251,2.274814329938806
MSFT,0.61561,0.493418,0.240662,1.8840491870700762
AMZN,0.585521,0.474082,0.320973,1.3523930586880877
TSLA,0.522713,0.432508,0.511584,0.7672407408772831
GOOGL,0.429502,0.367535,0.299436,1.0938417970812815
SOFI,0.344123,0.304191,0.653402,0.40433198410969445
AAPL,0.324918,0.289389,0.19921,1.2518922129152963
JPM,0.266688,0.24316,0.202744,1.0020536161175664
CAT,0.167189,0.159015,0.283609,0.41964480901025425
WMT,0.148209,0.142152,0.15365,0.6648377598756733


In [47]:
test_comparison = test_otpimized_bollinger_stocks_df[['Total Returns']].join(test_benchmark_boll_strat_df[['Total Returns']], how='outer', lsuffix='_Bollinger', rsuffix='_Benchmark')
test_comparison[test_comparison['Total Returns_Bollinger'] > test_comparison['Total Returns_Benchmark']]

,Total Returns_Bollinger,Total Returns_Benchmark
CVX,0.167788,-0.152793
JNJ,0.076446,-0.005895
NKE,0.025246,-0.170489
PFE,0.159964,-0.332988
SOFI,0.661432,0.344123
UNH,0.15314,0.089647
XOM,0.34098,-0.083341


### trading data codes

In [49]:
bollinger_all_best_params = {}

otpimized_bollinger_stocks_df = pd.DataFrame(index=stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])
for stock in stocks:
    best_std, best_window, best_total_return = bollinger_bayes_opt(data, stock, std_range=(1.0, 3.0), window_range=(10, 50), init_points = 5, n_iter = 50)
    metrics, _ = bollinger_bands_strat(data, stock, best_std, best_window)
    otpimized_bollinger_stocks_df.loc[stock, 'Total Returns'] = metrics['total_return']
    otpimized_bollinger_stocks_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    otpimized_bollinger_stocks_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    otpimized_bollinger_stocks_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

    bollinger_all_best_params[stock] = {"best_std": best_std,
                                        "best_window": best_window,
                                        "best_total_return": best_total_return}
    
    print(f'{stock} Optimal Parameters are std: {best_std:.3f}, window: {best_window}')

|   iter    |  target   |    std    |  window   |
-------------------------------------------------
| 1         | -0.02593  | 1.749     | 48.03     |
| 2         | -0.000989 | 2.464     | 33.95     |
| 3         | 0.04697   | 1.312     | 16.24     |
| 4         | -0.09945  | 1.116     | 44.65     |
| 5         | -0.008938 | 2.202     | 38.32     |
| 6         | -0.01399  | 2.986     | 15.09     |
| 7         | 0.02688   | 1.304     | 17.36     |
| 8         | 0.06801   | 1.038     | 14.13     |
| 9         | 0.04838   | 1.012     | 11.74     |
| 10        | 0.02946   | 1.005     | 26.23     |
| 11        | -0.01218  | 2.99      | 23.12     |
| 12        | 0.001838  | 2.977     | 29.06     |
| 13        | 0.0       | 2.976     | 10.14     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 14        | 0.03307   | 1.005     | 13.2      |
| 15        | 0.06716   | 1.033     | 14.97     |
| 16        | 0.09252   | 1.021     | 10.17     |
| 17        | -0.007087 | 1.005     | 20.41     |
| 18        | -0.02852  | 1.071     | 31.21     |
| 19        | -0.01218  | 2.989     | 26.39     |
| 20        | 0.09252   | 1.013     | 10.78     |
| 21        | 0.004585  | 2.951     | 41.07     |
| 22        | 0.0       | 2.988     | 49.98     |
| 23        | 0.04446   | 1.5       | 10.58     |
| 24        | -0.01399  | 2.974     | 18.63     |
| 25        | 0.02837   | 1.0       | 35.89     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 26        | 0.01449   | 1.079     | 24.16     |
| 27        | 0.04428   | 1.032     | 27.9      |
| 28        | 0.004585  | 2.991     | 36.09     |
| 29        | -0.04544  | 1.009     | 40.68     |
| 30        | 0.08523   | 1.005     | 10.36     |
| 31        | -0.02384  | 1.733     | 14.42     |
| 32        | 0.05069   | 1.001     | 11.16     |
| 33        | -0.01218  | 2.636     | 18.63     |
| 34        | 0.004585  | 2.84      | 41.97     |
| 35        | -0.00947  | 2.58      | 12.98     |
| 36        | 0.007142  | 1.034     | 15.58     |
| 37        | 0.09252   | 1.039     | 10.16     |
| 38        | -0.03243  | 1.972     | 16.56     |
| 39        | 0.06716   | 1.018     | 14.53     |
| 40        | 0.04056   | 1.299     | 28.04     |
| 41        | 0.01212   | 1.452     | 27.16     |
| 42        | -0.01218  | 2.771     | 16.0      |
| 43        | -0.02446  | 2.247     | 22.29     |
| 44        | 0.002385  | 1.453     | 12.26     |
| 45        | -0.01218  | 2.719     | 19.69     |


...

| 44        | 0.121     | 1.084     | 42.03     |
| 45        | 0.004375  | 2.719     | 19.69     |
| 46        | 0.0       | 2.61      | 11.14     |
| 47        | 0.1042    | 1.039     | 39.66     |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 48        | 0.1302    | 1.028     | 42.85     |
| 49        | -0.01518  | 1.731     | 42.66     |
| 50        | 0.1368    | 1.087     | 43.5      |


/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/166066661.py:31: RuntimeWarning: divide by zero encountered in scalar divide
  (annualized_return - rf) / annualized_volatility,


| 51        | 0.0       | 2.997     | 23.55     |
| 52        | 0.0       | 2.977     | 27.28     |
| 53        | -0.00621  | 1.624     | 34.66     |
| 54        | 0.0       | 2.962     | 13.87     |
| 55        | 0.05999   | 1.0       | 27.43     |
XOM Optimal Parameters are std: 1.007, window: 47


In [50]:
otpimized_bollinger_stocks_df = otpimized_bollinger_stocks_df.sort_values(by='Total Returns', ascending=False)
otpimized_bollinger_stocks_df

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
NVDA,0.542048,0.555886,0.218502,2.3610104159857555
TSLA,0.50922,0.577973,0.383597,1.4024431608716459
BA,0.330327,0.358757,0.170075,1.8742195834879567
GS,0.228747,0.26504,0.115303,1.9517194171359726
SOFI,0.209395,0.276385,0.248065,0.9529164647084498
AMZN,0.20157,0.255626,0.127265,1.6943159302271273
JPM,0.192201,0.232011,0.169017,1.1360419741266083
XOM,0.176589,0.24194,0.112776,1.7906384854703488
MSFT,0.15116,0.203582,0.103403,1.581982213713049
CAT,0.129106,0.153039,0.086721,1.303476584254131


In [51]:
benchmark_boll_strat_df = pd.DataFrame(index=selected_stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])

for stock in stocks:
    metrics, _ = benchmark_strategy(data, stock)
    
    benchmark_boll_strat_df.loc[stock, 'Total Returns'] = metrics['total_return']
    benchmark_boll_strat_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    benchmark_boll_strat_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    benchmark_boll_strat_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

benchmark_boll_strat_df = benchmark_boll_strat_df.sort_values(by='Total Returns', ascending=False)
benchmark_boll_strat_df

/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.DataFrame(index=data.index, columns=['Returns', 'Cumulative Returns']).fillna(0)
/var/folders/3w/vx714gsj5tdcyq6m356x9syr0000gn/T/ipykernel_12600/97245288.py:25: FutureWarning: Downcasting object dtype arrays 

,Total Returns,Annualized Return,Annualized Volatility,Sharpe Ratio
TSLA,1.042144,0.817855,0.651925,1.193165363819431
SOFI,0.813126,0.681605,0.526906,1.2176851222520606
NVDA,0.623839,0.555309,0.520659,0.9897245031679158
GS,0.606342,0.542899,0.271529,1.8520994449456636
WMT,0.572402,0.518438,0.178516,2.680077539635155
GOOGL,0.411764,0.394999,0.275298,1.289506528453523
JPM,0.403652,0.388398,0.246083,1.4157722521803078
AAPL,0.275108,0.278381,0.236102,1.0096521343371667
AMZN,0.238133,0.244674,0.277033,0.7388070234461415
CAT,0.143066,0.153164,0.26459,0.42769481524586483


In [52]:
comparison = otpimized_bollinger_stocks_df[['Total Returns']].join(benchmark_boll_strat_df[['Total Returns']], how='outer', lsuffix='_Bollinger', rsuffix='_Benchmark')
comparison[comparison['Total Returns_Bollinger'] > comparison['Total Returns_Benchmark']]

,Total Returns_Bollinger,Total Returns_Benchmark
BA,0.330327,-0.15535
JNJ,0.034974,-0.066744
MCD,0.056885,-0.02006
MSFT,0.15116,0.027594
NKE,0.107932,-0.292576
PFE,0.11113,0.041464
UNH,0.090728,0.059351
XOM,0.176589,0.077639


## SMA strategy

In [54]:
def sma_strat(data, stock, short_window, long_window, rf=0.04):
    stock_df = data[[f'{stock}']].copy()
    stock_df.rename(columns={f'{stock}': 'Close'}, inplace=True)
    stock_df = data[[stock]].copy()

    # calculate daily log returns
    stock_df['Log_Returns'] = np.log(stock_df[stock] / stock_df[stock].shift(1))

    # calculate moving averages
    stock_df['Short_SMA'] = stock_df[stock].rolling(window=short_window).mean()
    stock_df['Long_SMA'] = stock_df[stock].rolling(window=long_window).mean()
    stock_df.dropna(inplace=True)

    # generate signals: 1 for long, -1 for short, 0 for no position
    stock_df['Signal'] = np.where(stock_df['Short_SMA'] > stock_df['Long_SMA'], 1, 0)
    stock_df['Signal'] = np.where(stock_df['Short_SMA'] < stock_df['Long_SMA'], -0.5, stock_df['Signal'])

    # detect when a trade occurs (position change)
    stock_df['Trade'] = stock_df['Signal'].diff().fillna(0).abs() > 0

    # calculate strategy returns
    stock_df['SMA_Returns'] = stock_df['Log_Returns'] * stock_df['Signal'].shift(1)

    # total simple return
    total_return = np.exp(stock_df['SMA_Returns'].sum()) - 1

    # performance metrics
    annualized_return = np.mean(np.exp(stock_df['SMA_Returns']) - 1) * 252
    daily_volatility = np.std(np.exp(stock_df['SMA_Returns']) - 1)
    annualized_volatility = daily_volatility * np.sqrt(252)
    sharpe_ratio = np.where(annualized_volatility != 0,
                        (annualized_return - rf) / annualized_volatility,
                        0)

    metrics = {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_volatility,
        'sharpe_ratio': sharpe_ratio}

    return metrics, stock_df

In [55]:
def plot_sma(stock_df, stock):
    # plot stock price series, SMAs, trading actions
    plt.rcParams['figure.figsize'] = 12, 6
    plt.grid(True, alpha = .3)
    plt.plot(stock_df[stock], label = f'{stock}')
    plt.plot(stock_df['Short_SMA'], label = 'Short_SMA')
    plt.plot(stock_df['Long_SMA'], label = 'Long_SMA')
    plt.plot(stock_df[stock_df.Action == 2].index, stock_df['Short_SMA'][stock_df.Action == 2], '^', color = 'g', markersize = 12)
    plt.plot(stock_df[stock_df.Action == -1].index, stock_df['Short_SMA'][stock_df.Action == -2], 'v', color = 'r', markersize = 12)
    plt.legend(loc=1)
    plt.show()

    # plot cumulative returns (comparison with benchmark strategy (buy and hold)
    plt.plot(stock_df.index, np.exp(stock_df['SMA_Returns'].cumsum()), label= 'SMA Strategy')
    plt.plot(stock_df.index, np.exp(stock_df['Log_Returns'].cumsum()), label='Benchmark (Buy & Hold) Strategy')
    plt.title(f'{stock} Cumulative Returns: SMA Strategy vs Benchmark')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return')
    plt.legend()
    plt.show()

### bayes opt (non-rolling) for sma strategy

In [57]:
# find parameters values for short and long window size for SMA strat that maximises returns for each stock
def sma_bayes_opt(data, stock, short_range=(5, 50), long_range=(51, 200), init_points = 5, n_iter = 50):
    # bounds for the params in the opt process
    pbounds = {'short_window': short_range, 'long_window': long_range}

    def objective(short_window, long_window): # objective function for bayes opt
        metrics, _ = sma_strat(data, stock, int(short_window), int(long_window))
        return metrics['total_return']

    # perform bayes opt
    optimizer = BayesianOptimization(f=objective,  # target function to maximise
                                     pbounds=pbounds,  # params bounds
                                     random_state=42)

    # optimize the params
    optimizer.maximize(init_points=init_points, n_iter=n_iter)

    # extract optimized params and total return
    best_short = int(optimizer.max['params']['short_window']) # round to integer
    best_long = int(optimizer.max['params']['long_window'])  # round to integer
    best_total_return = optimizer.max['target']

    return best_short, best_long, best_total_return

### test data codes

In [ ]:
test_sma_all_best_params = {}

test_otpimized_sma_stocks_df = pd.DataFrame(index=stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])
for stock in stocks:
    best_short, best_long, best_total_return = sma_bayes_opt(test_data, stock, short_range=(5, 50), long_range=(51, 200), init_points = 5, n_iter = 50)
    metrics, _ = sma_strat(test_data, stock, best_short, best_long)
    test_otpimized_sma_stocks_df.loc[stock, 'Total Returns'] = metrics['total_return']
    test_otpimized_sma_stocks_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    test_otpimized_sma_stocks_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    test_otpimized_sma_stocks_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

    test_sma_all_best_params[stock] = {"best_short": best_short,
                                       "best_long": best_long,
                                       "best_total_return": best_total_return}
    
    print(f'{stock} Optimal Parameters are short: {best_short:.3f}, long: {best_long}')

|   iter    |  target   | long_w... | short_... |
-------------------------------------------------
| 1         | -0.1723   | 106.8     | 47.78     |
| 2         | -0.1102   | 160.1     | 31.94     |
| 3         | -0.01823  | 74.25     | 12.02     |
| 4         | 0.1246    | 59.65     | 43.98     |
| 5         | -0.0718   | 140.6     | 36.86     |
| 6         | 0.03419   | 51.18     | 32.72     |
| 7         | 0.123     | 59.78     | 42.82     |
| 8         | 0.05386   | 199.5     | 5.871     |
| 9         | 0.05386   | 199.9     | 49.57     |
| 10        | 0.004156  | 72.31     | 49.79     |
| 11        | 0.05386   | 199.9     | 27.06     |
| 12        | -0.1335   | 121.7     | 5.126     |
| 13        | 0.03466   | 51.74     | 49.51     |
| 14        | -0.007245 | 178.5     | 5.177     |
| 15        | -0.07812  | 51.02     | 5.481     |
| 16        | 0.0354    | 182.7     | 49.3      |
| 17        | 0.05872   | 187.2     | 33.18     |
| 18        | 0.08262   | 69.18     | 31.86     |


In [ ]:
test_otpimized_sma_stocks_df = test_otpimized_sma_stocks_df.sort_values(by='Total Returns', ascending=False)
test_otpimized_sma_stocks_df

In [ ]:
test_benchmark_sma_strat_df = pd.DataFrame(index=selected_stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])

for stock in stocks:
    metrics, _ = benchmark_strategy(test_data, stock)
    
    test_benchmark_sma_strat_df.loc[stock, 'Total Returns'] = metrics['total_return']
    test_benchmark_sma_strat_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    test_benchmark_sma_strat_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    test_benchmark_sma_strat_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

test_benchmark_sma_strat_df = test_benchmark_sma_strat_df.sort_values(by='Total Returns', ascending=False)
test_benchmark_sma_strat_df

In [ ]:
test_comparison_sma = test_otpimized_sma_stocks_df[['Total Returns']].join(test_benchmark_sma_strat_df[['Total Returns']], how='outer', lsuffix='_SMA', rsuffix='_Benchmark')
test_comparison_sma[test_comparison_sma['Total Returns_SMA'] > test_comparison_sma['Total Returns_Benchmark']]

### trading data codes

In [ ]:
sma_all_best_params = {}

otpimized_sma_stocks_df = pd.DataFrame(index=stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])
for stock in stocks:
    best_short, best_long, best_total_return = sma_bayes_opt(data, stock, short_range=(5, 50), long_range=(51, 200), init_points = 5, n_iter = 50)
    metrics, _ = sma_strat(data, stock, best_short, best_long)
    otpimized_sma_stocks_df.loc[stock, 'Total Returns'] = metrics['total_return']
    otpimized_sma_stocks_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    otpimized_sma_stocks_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    otpimized_sma_stocks_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

    sma_all_best_params[stock] = {"best_short": best_short,
                                  "best_long": best_long,
                                  "best_total_return": best_total_return}
    
    print(f'{stock} Optimal Parameters are short: {best_short:.3f}, long: {best_long}')

In [ ]:
otpimized_sma_stocks_df = otpimized_sma_stocks_df.sort_values(by='Total Returns', ascending=False)
otpimized_sma_stocks_df

In [ ]:
benchmark_sma_strat_df = pd.DataFrame(index=selected_stocks, columns=['Total Returns', 'Annualized Return', 'Annualized Volatility', 'Sharpe Ratio'])

for stock in stocks:
    metrics, _ = benchmark_strategy(data, stock)
    
    benchmark_sma_strat_df.loc[stock, 'Total Returns'] = metrics['total_return']
    benchmark_sma_strat_df.loc[stock, 'Annualized Return'] = metrics['annualized_return']
    benchmark_sma_strat_df.loc[stock, 'Annualized Volatility'] = metrics['annualized_volatility']
    benchmark_sma_strat_df.loc[stock, 'Sharpe Ratio'] = metrics['sharpe_ratio']

benchmark_sma_strat_df = benchmark_sma_strat_df.sort_values(by='Total Returns', ascending=False)
benchmark_sma_strat_df

In [ ]:
comparison_sma = otpimized_sma_stocks_df[['Total Returns']].join(benchmark_sma_strat_df[['Total Returns']], how='outer', lsuffix='_SMA', rsuffix='_Benchmark')
comparison_sma[comparison_sma['Total Returns_SMA'] > comparison_sma['Total Returns_Benchmark']]

## portfolio optimisation (mvo, maximise sharpe ratio)

In [ ]:
def mean_variance_optimisation(returns_df, rf=0.04): # using test returns, allowed short selling, max leverage = 2
    
    # calculate expected annualized returns for each stock
    expected_returns = returns_df.mean() * 252

    # calculate the annualized covariance matrix of returns
    cov_matrix = eturns_df.cov() * 252

    # objective function (to minimise negative sharpe ratio)
    def objective(weights):
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = (portfolio_return - rf) / portfolio_volatility
        return -sharpe_ratio

    # initial guess (equal weights)
    num_assets = 10
    init_guess = num_assets * [1. / num_assets]

    # constraints: weights sum to 1
    constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
                  {'type': 'ineq', 'fun': lambda weights: 2 - np.sum(np.abs(weights))})  # max leverage = 2 -> total exposure (long + short positions) of the portfolio cannot exceed 200% of the portfolio’s capital

    # optimization using Sequential Least Squares Quadratic Programming (SLSQP)
    result = minimize(objective, init_guess, method='SLSQP', constraints=constraints)

    if not result.success:
        raise BaseException(result.message)

    optimal_weights = result.x
    max_sharpe_ratio = -result.fun

    # Store the optimal weights in a DataFrame
    optimal_weights_df = pd.DataFrame(optimal_weights, index=returns_df.columns, columns=['Optimal Weights'])

    return optimal_weights_df, max_sharpe_ratio

In [ ]:
optimal_weights_df, max_sharpe_ratio = mvo(data, SMA_all_best_params, risk_free_rate=0.04)

In [ ]:
optimal_weights_df

In [ ]:
max_sharpe_ratio